
## Trial to combine middle ear and bruxism tagging to identify pure moments of middle ear activation

In [1]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')

In [15]:

import mne
from tinnsleep.reports import combine_brux_MEMA
from tinnsleep.data import CreateRaw, CleanAnnotations, AnnotateRaw_sliding
from tinnsleep.events.scoring import classif_to_burst, burst_to_episode, create_list_events
import warnings
import matplotlib.pyplot as plt
%matplotlib qt
import pandas as pd
from tinnsleep.config import Config

print("Config loaded")


Config loaded


## Loading parameters and results

In [40]:


# change to --overwrite True if you want to force to recompute the results
%run ../compute_results.py --bruxism True --mema True --overwrite False 

mema_files= pd.read_csv("data/mema_files.csv",engine='python', sep="; ")["files_with_mema"].values

results_MEMA = pd.read_pickle("data/reports_and_datas_MEMA.pk").to_dict()
results_brux = pd.read_pickle("data/reports_and_datas_bruxism.pk").to_dict()
min_burst_joining_brux = 3 
min_burst_joining_MEMA = 0
EDF_list = Config.bruxisme_files[:30]
print(EDF_list)

config loaded.
Performs Bruxism: <True>
Performs MEMA: <True>
Will overwrite existing results: <False>
parameters set
result files exist: Reports creation skipped.
['E:/Acou_sommeil/EDF_V2_PAUL\\1BA07_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1BA07_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1CC05_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1CC05_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1DA15_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1DA15_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1DL12_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1DL12_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1GB18_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1GB19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1GF14_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1GF14_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1MA16_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1MA16_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1MF19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1MF19_nuit_son.edf', 'E:/Acou_sommeil/ED

## Bruxism + MEMA processing for pure MEMA visualisation

In [37]:
from time import time
# load file from config

start = time()

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    print("Files processed : ")
    
    #Loop on all the patient files
    for filename in EDF_list:
        #opens the raw file
        file = filename.split(os.path.sep)[-1]
        print(file, end=" ")
        if not (file in results_brux.keys() and file in results_MEMA.keys()):
            print(results_brux[file]["parameters"]["time_interval"])
            print(f"(does not have both bruxism and mema)... skipping")
        else:
            window_length_brux = results_brux[file]["parameters"]["time_interval"]
            delim_brux = results_brux[file]["parameters"]["delim"]
            window_length_MEMA = results_MEMA[file]["parameters"]["time_interval"]
            delim_MEMA = results_MEMA[file]["parameters"]["delim"]
            params_combine = dict(
                                labels_brux = results_brux[file]["labels"][0], 
                                labels_artifacts_brux = results_brux[file]["parameters"]["valid_labels"], 
                                time_interval_brux = results_brux[file]["parameters"]["time_interval"], 
                                delim_ep_brux = results_brux[file]["parameters"]["delim"], 
                                labels_MEMA = results_MEMA[file]["labels"][0],
                                labels_artifacts_MEMA = results_MEMA[file]["parameters"]["valid_labels"],
                                time_interval_MEMA = results_MEMA[file]["parameters"]["time_interval"],
                                delim_ep_MEMA = results_brux[file]["parameters"]["delim"],
                                min_burst_joining_brux=min_burst_joining_brux, 
                                min_burst_joining_MEMA=min_burst_joining_MEMA
                                )
            print(results_MEMA[file]["parameters"]["time_interval"])
            print(len(results_brux[file]["labels"][0]))
            print(np.sum(results_brux[file]["labels"][0]))
            print(results_brux[file]["reports"][0])
            print(len(results_MEMA[file]["labels"][0])*4)
            print(results_brux[file]["parameters"]["time_interval"])
            brux_comb_ep, brux_pure_ep, compt_arti_brux, MEMA_comb_ep, MEMA_pure_ep, compt_arti_MEMA = combine_brux_MEMA(**params_combine)
            print("heu")
            print(len(MEMA_comb_ep))
            print(np.sum(brux_comb_ep))
            print(np.sum(brux_pure_ep))
            print(window_length_brux)
            print(delim_brux)
            print(min_burst_joining_brux)
            
            pure_brux_bursts = classif_to_burst(brux_pure_ep, time_interval=window_length_brux)
            li_ep_brux_p = burst_to_episode(pure_brux_bursts, delim=delim_brux, min_burst_joining= min_burst_joining_brux)
            pure_brux_events = create_list_events(li_ep_brux_p, window_length_brux, len(brux_pure_ep) * window_length_brux)
            
            comb_brux_bursts = classif_to_burst(brux_comb_ep, time_interval=window_length_brux)
            li_ep_brux_c = burst_to_episode(comb_brux_bursts, delim=delim_brux, min_burst_joining= min_burst_joining_brux)
            comb_brux_events = create_list_events(li_ep_brux_c, window_length_brux, len(brux_comb_ep) * window_length_brux)
            
            comb_MEMA_bursts = classif_to_burst(MEMA_comb_ep, time_interval=window_length_brux) # window_length_brux is valid ?
            
            pure_MEMA_bursts = classif_to_burst(MEMA_pure_ep, time_interval=window_length_brux) # window_length_brux is valid ?
            
            print("rapport d'activité : ")
            print("nb episodes bruxisme combinés : ")
            print(len(li_ep_brux_c))
            print("Nb épisodes bruxisme purs : ")
            print(len(li_ep_brux_p))
            print("nb episodes MEMA combinés : ")
            print(len(comb_MEMA_bursts))
            print("Nb épisodes MEMA purs : ")
            print(len(pure_MEMA_bursts))
            
            print("rapport d'activité taux horaire : ")
            print("nb episodes bruxisme combinés : ")
            print(len(li_ep_brux_c)/ ((len(MEMA_pure_ep)*window_length_brux)/3600))
            print("Nb épisodes bruxisme purs : ")
            print(len(li_ep_brux_p)/ ((len(MEMA_pure_ep)*window_length_brux)/3600))
            print("nb episodes MEMA combinés : ")
            print(len(comb_MEMA_bursts)/ ((len(MEMA_pure_ep)*window_length_brux)/3600))
            print("Nb épisodes MEMA purs : ")
            print(len(pure_MEMA_bursts)/ ((len(MEMA_pure_ep)*window_length_brux)/3600))
            
            for i in range(len(MEMA_comb_ep)):
                if MEMA_comb_ep[i]>0:
                    MEMA_comb_ep[i]+=3
            
            for i in range(len(MEMA_pure_ep)):
                if MEMA_pure_ep[i]>0:
                    MEMA_pure_ep[i]+=2
                    
            for i in range(len(comb_brux_events)):
                if comb_brux_events[i]>0:
                    comb_brux_events[i]+=1
                    
            plt.figure()
            plt.plot(pure_brux_events, label="pure_brux")
            plt.plot(comb_brux_events, label="comb_brux")
            plt.plot(MEMA_comb_ep, label="comb_MEMA")
            plt.plot(MEMA_pure_ep, label="pure_MEMA")
            plt.legend()
            plt.title("Where the events happen for " + file)
                        
                       

Files processed : 
1BA07_nuit_hab.edf 0.248
(does not have both bruxism and mema)... skipping
1BA07_nuit_son.edf 0.245
(does not have both bruxism and mema)... skipping
1CC05_nuit_hab.edf 0.244
(does not have both bruxism and mema)... skipping
1CC05_nuit_son.edf 0.24
(does not have both bruxism and mema)... skipping
1DA15_nuit_hab.edf 0.96
122624
15528
{'Clean data duration': 23940.48, 'Total burst duration': 3726.72, 'Total number of burst': 2072, 'Number of bursts per hour': 311.5727002967359, 'Total number of episodes': 288, 'Number of bursts per episode': 7.194444444444445, 'Number of episodes per hour': 43.307402357847465, 'Number of tonic episodes per hour': 15.63878418477825, 'Number of phasic episodes per hour': 2.0931911139626274, 'Number of mixed episodes per hour': 4.547277247573984, 'Mean duration of tonic episode': 6.173076923076987, 'Mean duration of phasic episode': 6.475862068965398, 'Mean duration of mixed episode': 26.902857142857226, 'Power Ratio': array([24.40177746

KeyError: '1GF14_nuit_hab.edf'

C:\Users\Zeta\Anaconda3\lib\site-packages\ipykernel\eventloops.py:106: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  app.exec_()


In [11]:
print(len(results_brux[file]["labels"][0])*50)
print(len(results_MEMA[file]["labels"][0])*50*4)
print (len(MEMA_comb_ep)*50)
print (len(MEMA_pure_ep)*50)
print (len(brux_comb_ep)*50)

6627000
6627000
6627000
6627000
6627000


In [33]:
labels_int= results_brux[file]["labels"][0]
print(file)
print(filename)
duration_brux = 200*0.24
print(np.sum(labels_int))
print(len(labels_int)*48)

interval_brux = duration_brux
print (len(MEMA_comb_ep))
#print(len(raw))
#-----------------Pure MEMA episodes visualisation and comparison with ATM activity -----------------------------------
#Preparing raw for visualisation
picks_chan = ['Airflow', '1', '2', "Activity"]           # subset of EMG electrodes
raw  = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
print(raw.info["sfreq"])

raw  = CreateRaw(raw[picks_chan][0], raw.info["sfreq"], picks_chan, ch_types=['misc','emg','emg', "misc"])        # pick channels and load
print(len(raw))
scalings = dict(misc=1, emg=1e-5)

raw  = raw.filter(20., 99., n_jobs=4, 
                  fir_design='firwin', filter_length='auto', phase='zero-double',
                  picks=['1', '2'])
#Annotating the raw
raw = CleanAnnotations(raw)
dict_annotations = {1: "b_pure", 2:"b_comb", 3:"M_pure", 4:"M_comb"}
raw = AnnotateRaw_sliding(raw, labels_int , 
                dict_annotations=dict_annotations, duration=duration_brux, interval=interval_brux)


print("Raw annotated")
raw.plot(scalings = scalings)
plt.title(filename)
                    

1GB19_nuit_hab.edf
E:/Acou_sommeil/EDF_V2_PAUL\1GB19_nuit_hab.edf
4702
6361920
132540
200.0


<ipython-input-33-31ece4cce161>:14: RuntimeWarning: 6 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw  = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading


6362000
Raw annotated


Text(0.5, 1, 'E:/Acou_sommeil/EDF_V2_PAUL\\1GB19_nuit_hab.edf')